In [5]:
# B3 - REPUTATION COMPUTATION ALGORITHM & UNIT TESTS #
import random, time
# ------------------------------------------------------------------
# resolve_delegations(): Resolves chains to final voters with path compression
# ------------------------------------------------------------------
def resolve_delegations(delegation):
    final = {}
    for u in delegation:
        # walk from u until you hit a terminal or a memoized node
        path = []
        v = u
        while v not in final and delegation.get(v) is not None:
            path.append(v)
            v = delegation[v]
        # v is now either in final or a terminal (delegation[v] is None)
        root = final.get(v, v)
        # path–compress
        for x in path:
            final[x] = root
        # also remember u itself (in case path was empty)
        final[u] = root
    return final

# ------------------------------------------------------------------
# penalized_pagerank(): PageRank + Selective NOT_VOTED penalty
# ------------------------------------------------------------------
def penalized_pagerank(delegation, *, alpha = 0.15, tol = 1e-6, max_iter = 100,
                     penalty = 0.5, final_holders = None, votes = None):
    """delegation  : { user -> delegate OR None }  (out-degree ≤ 1)
    final_holders  : mapping user -> final_holder  (from Section 6.1)
    votes          : mapping final_holder -> choice  (those who voted)
    penalty        : fraction by which to reduce score for non-voting delegates
                    who held someone ELSE's token.  If final_holders or votes
                    are omitted, the function behaves like plain PageRank.
    returns        : { user -> reputation score }  (scores sum to 1.0) """

    # ----- PageRank core -----
    users = list(delegation.keys())
    N     = len(users)
    idx   = {u: i for i, u in enumerate(users)}
    # successor array: self-loop if sink
    succ  = [idx[u] for u in users]
    for u, v in delegation.items():
        if v is not None:
            succ[idx[u]] = idx[v]

    rank = [1.0/N] * N
    for _ in range(max_iter):
        new = [alpha / N] * N
        for j, nxt in enumerate(succ):
            new[nxt] += (1-alpha) * rank[j]
        if max(abs(a-b) for a, b in zip(rank, new)) < tol:
            break
        rank = new
    pr = {users[i]: rank[i] for i in range(N)}

    # ----- Selective penalty phase -----
    if final_holders is not None and votes is not None:
        # token counts per holder + external tokens received
        total_tok  = {}
        external   = {}
        for u, h in final_holders.items():
            total_tok[h]  = total_tok.get(h, 0) + 1
            if u != h:                          # token came from someone else
                external[h] = external.get(h, 0) + 1
        for h, score in pr.items():
            # penalise only if  (a) delegate missed vote AND (b) had externals
            if h not in votes and external.get(h, 0) > 0:
                frac = external[h] / total_tok[h]      # share of tokens lost
                pr[h] = score * (1 - penalty * frac)
        # renormalise
        s = sum(pr.values()) or 1
        for h in pr:
            pr[h] /= s
    return pr

# ------------------------------------------------------------------
# 3. Verbose test-suite
# ------------------------------------------------------------------
def show_scores(title, scores):
    top = ", ".join(f"{u}:{s:.3f}" for u, s in
                    sorted(scores.items(), key=lambda x: -x[1])[:5])
    print(f"{title:<30} {top}")

# (a) Star graph: everyone votes
deleg_a = {'A':'D','B':'D','C':'D','D':None}
final_a = resolve_delegations(deleg_a)
votes_a = {'D':'FOR'}
print("Test (a) star:")
raw_a = penalized_pagerank(deleg_a)
pen_a = penalized_pagerank(deleg_a, final_holders=final_a, votes=votes_a)
show_scores("  before penalty:", raw_a)
show_scores("  after  penalty:", pen_a)

# (b) Chain: Z fails to vote
deleg_b = {'X':'Y','Y':'Z','Z':None}
final_b = resolve_delegations(deleg_b)
votes_b = {'Y':'FOR'}  # Z did not vote
print("\nTest (b) chain (and Z not voted):")
raw_b = penalized_pagerank(deleg_b)
pen_b = penalized_pagerank(deleg_b, final_holders=final_b, votes=votes_b)
show_scores("  before penalty:", raw_b)
show_scores("  after  penalty:", pen_b)

# (c) Mixed scenario: R fails to vote
deleg_c = {'P':'Q','Q':'R','R':None,'S':'Q','T':None}
final_c = resolve_delegations(deleg_c)
votes_c = {'Q':'FOR','T':'AGAINST'}  # R did not vote
print("\nTest (c) mixed (and R not voted):")
raw_c = penalized_pagerank(deleg_c)
pen_c = penalized_pagerank(deleg_c, final_holders=final_c, votes=votes_c)
show_scores("  before penalty:", raw_c)
show_scores("  after  penalty:", pen_c)

# (d) Random 100 000-node graph
users = [f'U{i}' for i in range(100000)]
deleg_d = {u: None for u in users}
for u in users:
    if random.random() < 0.5:
        idx = users.index(u)
        if idx:
            deleg_d[u] = random.choice(users[:idx])

# Resolve final holders and pick ~60% to vote “FOR”
final_d = resolve_delegations(deleg_d)
holders = list(set(final_d.values()))
votes_d = {h: 'FOR' for h in random.sample(holders, int(0.6 * len(holders)))}

# Measure and report only convergence time
t0 = time.time()
_ = penalized_pagerank(deleg_d, final_holders=final_d, votes=votes_d)
print(f"\nTest (d): 100 000-node graph resolved in {time.time()-t0:.3f}s")

print("\nAll reputation computaion tests passed.")

Test (a) star:
  before penalty:              D:0.888, A:0.037, B:0.037, C:0.037
  after  penalty:              D:0.888, A:0.037, B:0.037, C:0.037

Test (b) chain (and Z not voted):
  before penalty:              Z:0.857, Y:0.092, X:0.050
  after  penalty:              Z:0.800, Y:0.130, X:0.070

Test (c) mixed (and R not voted):
  before penalty:              R:0.659, T:0.200, Q:0.081, P:0.030, S:0.030
  after  penalty:              R:0.547, T:0.266, Q:0.108, P:0.040, S:0.040

Test (d): 100 000-node graph resolved in 0.834s

All reputation computaion tests passed.
